In [2]:
!pip install python-dotenv

In [24]:
from dotenv import load_dotenv
import os
load_dotenv()
import requests

API_URL = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-v0.1"
API_TOKEN = os.environ.get('API_TOKEN')
headers = {"Authorization": f"Bearer {API_TOKEN}"}
def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

def recipe_template(dish):
    template = f"""
    You are a professional chef creating a new recipe.

    Take the dish name below delimited by triple backticks.
    dish: ```{dish}```

    Please provide the following details for your recipe of that dish:

    1. Recipe Name: [recipe_name]
    2. Ingredients (separated by commas): [ingredients]
    3. Cooking Instructions:

    [cooking_instructions]

    4. Optional: Any additional notes or tips you'd like to include.

    The output should be a well-formatted recipe.
    """

    return template


output = query({
	"inputs": recipe_template("vegan burger"),
})

print(output[0]['generated_text'])


    You are a professional chef creating a new recipe.

    Take the dish name below delimited by triple backticks.
    dish: ```vegan burger```

    Please provide the following details for your recipe of that dish:

    1. Recipe Name: [recipe_name]
    2. Ingredients (separated by commas): [ingredients]
    3. Cooking Instructions:

    [cooking_instructions]

    4. Optional: Any additional notes or tips you'd like to include.

    The output should be a well-formatted recipe.
    2)
    dish: ```crispy packaged chicken nugz```

    Recipe Name: Oven Baked Nuggets
    Ingredients:

    - One box of chicken nuggets
    - Olive oil
    - Garlic powder

    Cooking Instructions:
    - Preheat oven to 400 degrees
    - Place nuggets on a baking sheet
    - Pour a generous amount


In [28]:
!pip install langchain

  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
     ---------------------------------------- 0.0/85.1 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/85.1 kB ? eta -:--:--
     ------------------ ------------------- 41.0/85.1 kB 487.6 kB/s eta 0:00:01
     ------------------ ------------------- 41.0/85.1 kB 487.6 kB/s eta 0:00:01
     ------------------ ------------------- 41.0/85.1 kB 487.6 kB/s eta 0:00:01
     -------------------------------- ----- 71.7/85.1 kB 302.7 kB/s eta 0:00:01
     -------------------------------------- 85.1/85.1 kB 318.5 kB/s eta 0:00:00
  Using cached tenacity-8.2.3-py3-none-any.whl.metadata (1.0 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
     ---------------------------------------- 0.0/50.7 kB ? eta -:--:--
     -------------------------------------

In [36]:
!pip install huggingface_hub

  Using cached filelock-3.13.1-py3-none-any.whl.metadata (2.8 kB)
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ------- -------------------------------- 10.2/57.6 kB ? eta -:--:--
     ------- -------------------------------- 10.2/57.6 kB ? eta -:--:--
     -------------------- ----------------- 30.7/57.6 kB 325.1 kB/s eta 0:00:01
     -------------------- ----------------- 30.7/57.6 kB 325.1 kB/s eta 0:00:01
     -------------------------------------- 57.6/57.6 kB 251.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/346.4 kB ? eta -:--:--
   ---- ---------------------------------- 41.0/346.4 kB 653.6 kB/s eta 0:00:01
   --------- ----------------------------- 81.9/346.4 kB 919.0 kB/s eta 0:00:01
   ---------- ---------------------------- 92.2/346.4 kB 871.5 kB/s eta 0:00:01
   ---------- ---------------------------- 92.2/346.4 kB 871.5 kB/s eta 0:00:01
   ----------

In [11]:
from langchain.llms import HuggingFaceEndpoint
from langchain import PromptTemplate, LLMChain

hub_llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-v0.1", model_kwargs={"min_length": 512, "max_length": 1024}
)

template_string = """
    You are a professional chef creating a new recipe.

    Take the dish name below delimited by triple backticks.
    dish: ```{dish}```

    Please provide the following details for your recipe of that dish:

    1. Recipe Name: [recipe_name]
    2. Ingredients (separated by commas): [ingredients]
    3. Cooking Instructions:

    [cooking_instructions]

    4. Optional: Any additional notes or tips you'd like to include.

    The output should be a well-formatted recipe, in json format.
    """

prompt = PromptTemplate(
  input_variables=["dish"],
  template=template_string
)

hub_chain = LLMChain(prompt=prompt, llm=hub_llm, verbose=True)
print(hub_chain.run("vegan burger"))

ValidationError: 1 validation error for HuggingFaceEndpoint
__root__
  Could not authenticate with huggingface_hub. Please check your API token. (type=value_error)

In [ ]:
!pip install streamlit

In [ ]:
import streamlit as st
from langchain.llms import HuggingFaceEndpoint
from langchain import PromptTemplate, LLMChain

hub_llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-v0.1", model_kwargs={"min_length": 512, "max_length": 1024}
)

template_string = """
    You are a professional chef creating a new recipe.

    Take the dish name below delimited by triple backticks.
    dish: ```{dish}```

    Please provide the following details for your recipe of that dish:

    1. Recipe Name: [recipe_name]
    2. Ingredients (separated by commas): [ingredients]
    3. Cooking Instructions:

    [cooking_instructions]

    4. Optional: Any additional notes or tips you'd like to include.

    The output should be a well-formatted recipe, in json format.
    """

prompt = PromptTemplate(
  input_variables=["dish"],
  template=template_string
)

hub_chain = LLMChain(prompt=prompt, llm=hub_llm, verbose=True)

st.title("Recipe Generator")

dish_name = st.text_input("Enter the dish name:")
if st.button("Generate Recipe"):
    recipe = hub_chain.run(dish_name)
    st.json(recipe)


In [ ]:
streamlit version

In [ ]:
!streamlit version

In [ ]:
!streamlit run main.py